In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Bidirectional, Dropout, LSTM, Dense
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from google.colab import files, drive
drive.mount('/content/drive')
file_path_train = "/content/drive/MyDrive/Online Hackathon/Cryptocurrency Price Prediction/train.csv"
file_path_test = "/content/drive/MyDrive/Online Hackathon/Cryptocurrency Price Prediction/test.csv"
# โหลดข้อมูล
train_data = pd.read_csv(file_path_train)
test_data = pd.read_csv(file_path_test)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Step 2: เตรียมข้อมูล
train_prices = train_data['price'].values

In [ ]:
# Step 3: ปรับขนาดข้อมูลด้วย MinMaxScaler (ทำให้ค่าอยู่ในช่วง 0-1)
scaler = MinMaxScaler(feature_range=(0, 1))
train_prices_scaled = scaler.fit_transform(train_prices.reshape(-1, 1))

# Step 4: สร้างข้อมูลลำดับเวลา (sequence) สำหรับการฝึกโมเดล LSTM
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 60  # ใช้ 60 วันก่อนหน้านี้ในการทำนาย
X_train, y_train = create_dataset(train_prices_scaled, time_step)

# Reshape X เป็น [จำนวนตัวอย่าง, time_step, features]
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

# Step 5: สร้างโมเดล LSTM
model = Sequential()
model.add(Bidirectional(LSTM(units=100, return_sequences=True, input_shape=(X_train.shape[1], 1))))
model.add(Dropout(0.2))  # Dropout เพื่อป้องกัน overfitting
model.add(Bidirectional(LSTM(units=100, return_sequences=False)))
model.add(Dropout(0.2))  # Dropout เพิ่มป้องกัน overfitting
model.add(Dense(units=1))

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Step 6: ใช้ EarlyStopping เพื่อลดการ overfitting และฝึกโมเดล
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# เทรนโมเดล
model.fit(X_train, y_train, epochs=3, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Step 7: ใช้ข้อมูล train เพื่อทำนายค่าที่ต้องการส่งออก
# ใช้ข้อมูลล่าสุดจาก train_data (60 ตัวอย่างล่าสุด)
last_data = train_prices_scaled[-time_step:].reshape(1, -1)
last_data = last_data.reshape((last_data.shape[0], last_data.shape[1], 1))

# Step 8: ทำนายราคาจากข้อมูลทดสอบที่ไม่เคยเห็น
predicted_prices = []

# สร้างอาเรย์ที่ใช้ในการทำนายราคาทั้งหมด
for i in range(len(test_data)):
    # ทำนายราคาของเหรียญในทุก ๆ ID โดยไม่ต้องทำนายทีละตัว
    prediction = model.predict(last_data)

    # บันทึกผลทำนาย
    predicted_prices.append(prediction[0][0])

    # อัพเดตข้อมูลสำหรับทำนายครั้งถัดไป
    last_data = np.roll(last_data, shift=-1, axis=1)  # Shift ข้อมูลในแนวนอน
    last_data[0, -1, 0] = prediction  # ใส่ค่าทำนายในตำแหน่งสุดท้าย

# Convert predicted prices back to the original scale
predicted_prices_rescaled = scaler.inverse_transform(np.array(predicted_prices).reshape(-1, 1))

# Step 9: สร้างผลลัพธ์ในรูปแบบที่ต้องการ
submission = pd.DataFrame({'id': test_data['id'], 'price': predicted_prices_rescaled.flatten()})

# ปรับราคาทำนายให้มี 3 ตำแหน่งทศนิยม
submission['price'] = submission['price'].round(3)

# Step 11: ส่งออกผลลัพธ์
submission.to_csv('submission.csv', index=False)

Epoch 1/3


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


149/149 ━━━━━━━━━━━━━━━━━━━━ 85s 231ms/step - loss: 0.0152 - val_loss: 8.8413e-04
Epoch 2/3
149/149 ━━━━━━━━━━━━━━━━━━━━ 36s 239ms/step - loss: 0.0014 - val_loss: 4.9374e-04
Epoch 3/3
149/149 ━━━━━━━━━━━━━━━━━━━━ 43s 251ms/step - loss: 0.0011 - val_loss: 3.4154e-04
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 621ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step


<ipython-input-80-69085b207dab>:53: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  last_data[0, -1, 0] = prediction  # ใส่ค่าทำนายในตำแหน่งสุดท้าย


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━